<a href="https://colab.research.google.com/github/daegonYu/ko-gpts/blob/master/polyglot_ko_12_8b%EB%AA%A8%EB%8D%B8_%EB%82%98%EC%9D%98%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%95%99%EC%8A%B5_%EC%A0%84%ED%9B%84%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [ ]:
!nvidia-smi

Tue Jul 25 05:03:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [ ]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df

,train
0,{'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인...
1,"{'instruction': '스웨터의 유래는 어디에서 시작되었나요?', 'outp..."
2,{'instruction': '토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? ...
3,{'instruction': '화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?...
4,"{'instruction': ''사이보그'는 언제 처음 등장한 말이며, 그 의미와 ..."
...,...
21150,"{'instruction': '왕의 우물이란 무엇인가요?', 'output': '왕..."
21151,{'instruction': '한 달의 첫째 주가 어떻게 정의되는가? 매달 첫째주가...
21152,{'instruction': '산모와 태아가 혈액형이 다를 때 문제가 없는 이유는 ...
21153,"{'instruction': '의자의 쇠 부분을 만지면 왜 정전기가 발생할까요?',..."


In [ ]:
df.loc[:5]['train']

0    {'instruction': '양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인...
1    {'instruction': '스웨터의 유래는 어디에서 시작되었나요?', 'outp...
2    {'instruction': '토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? ...
3    {'instruction': '화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?...
4    {'instruction': ''사이보그'는 언제 처음 등장한 말이며, 그 의미와 ...
5    {'instruction': '씨앗을 냉장 처리후 심는 이유와 장기보존하려면 어떻게...
Name: train, dtype: object

In [ ]:
# # data
# data = data.map(
#     lambda x:
#     {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
#     if x['input'] else
#     {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
# )
# data
data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})

In [ ]:
data['train'][:5]

{'instruction': ['양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?',
  '스웨터의 유래는 어디에서 시작되었나요?',
  '토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?  \n\n토성의 고리는 얼음과 같은 여러 물질로 이루어져 있다고 알고 있는데, 카시니가 찍은 사진에서 마치 빛의 띠 처럼 보이는 이유가 무엇인가요? 물질의 공전 속도가 빠르기 때문에 카메라로 담았을 때 빛의 궤적으로 보이는 건가요? 또한, 야간에 빠르게 움직이는 자동차를 장노출로 찍었을 때 빛의 궤적이 생기는 것과 같은 원리일까요? 그리고 빛의 궤적이 생기는 것은 우주라는 어두운 환경 특성 때문이라고 생각됩니다. 이게 맞을까요?',
  '화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런칭을 위해 OEM과 ODM용어에 대해 혼란스러움을 느끼고 있습니다. 두 용어의 차이점이 무엇인지 알고 싶습니다.',
  "'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고 싶습니다."],
 'output': ['양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\n식물의 부위의 구분에 대해 궁금해하는 분이라면 분명 이 질문에 대한 답을 찾고 있을 것입니다. 양파는 잎이 아닌 줄기 부분입니다. 고구마는 다른 질문과 답변에서 언급된 것과 같이 뿌리 부분입니다. 따라서, 양파는 식물의 줄기 부분이 되고, 고구마는 식물의 뿌리 부분입니다.\n\n 덧붙이는 답변: 고구마 줄기도 볶아먹을 수 있나요? \n\n고구마 줄기도 식용으로 볶아먹을 수 있습니다. 하지만 줄기 뿐만 아니라, 잎, 씨, 뿌리까지 모든 부위가 식용으로 활용되기도 합니다. 다만, 한국에서는 일반적으로 뿌리 부분인 고구마를 주로 먹습니다.',
  '스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 

data 조작

In [ ]:
from datasets import Dataset, DatasetDict
from datasets import concatenate_datasets, load_dataset

# 기존의 데이터
existing_data = data['train'][:5]
existing_data = Dataset.from_dict(existing_data)

print(existing_data)

# 새로운 데이터
new_data = {
    'instruction': ['닥쳐줄래?','닥쳐줄래?','닥쳐줄래?'],
    'output': ['네! 닥치겠습니다.','네! 닥치겠습니다.','네! 닥치겠습니다.'],
    'url': ['new_url','new_url','new_url'],
}

new_data = Dataset.from_dict(new_data)


print(new_data)


Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 5
})
Dataset({
    features: ['instruction', 'output', 'url'],
    num_rows: 3
})


In [ ]:
dataset_dict = concatenate_datasets([existing_data, new_data])

dataset_dict = dataset_dict.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

print(dataset_dict)


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'output', 'url', 'text'],
    num_rows: 8
})


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

학습 전 추론

In [ ]:
tmp = pd.DataFrame(dataset_dict)
tmp

,instruction,output,url,text
0,양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?,양파는 잎이 아닌 식물의 줄기 부분입니다. 고구마는 식물의 뿌리 부분입니다. \n\...,https://kin.naver.com/qna/detail.naver?d1id=11...,### 질문: 양파는 어떤 식물 부위인가요? 그리고 고구마는 뿌리인가요?\n\n##...
1,스웨터의 유래는 어디에서 시작되었나요?,스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 ...,https://kin.naver.com/qna/detail.naver?d1id=11...,### 질문: 스웨터의 유래는 어디에서 시작되었나요?\n\n### 답변: 스웨터의 ...
2,토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 고리는 얼음과 ...,"토성의 고리가 미세한 입자들로 이루어져 있기 때문에, 입자들의 밀도 차이 때문에 카...",https://kin.naver.com/qna/detail.naver?d1id=11...,### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요? \n\n토성의 ...
3,화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 제조 브랜드 런...,화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에...,https://kin.naver.com/qna/detail.naver?d1id=5&...,### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?\n화장품 자체 ...
4,"'사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것이 있는지 알고...","'사이보그'는 1960년에 처음 등장한 말로, 기계와 유기체가 합성되어 생겨난 새로...",https://kin.naver.com/qna/detail.naver?d1id=11...,"### 질문: '사이보그'는 언제 처음 등장한 말이며, 그 의미와 종류에는 어떤 것..."
5,닥쳐줄래?,네! 닥치겠습니다.,new_url,### 질문: 닥쳐줄래?\n\n### 답변: 네! 닥치겠습니다.<|endoftext|>
6,닥쳐줄래?,네! 닥치겠습니다.,new_url,### 질문: 닥쳐줄래?\n\n### 답변: 네! 닥치겠습니다.<|endoftext|>
7,닥쳐줄래?,네! 닥치겠습니다.,new_url,### 질문: 닥쳐줄래?\n\n### 답변: 네! 닥치겠습니다.<|endoftext|>


In [ ]:
gen('닥쳐줄래?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 닥쳐줄래?

### 답변:....... 닥치거라.....-ㅂ- ### 질문: 말만하면 왜 ㅈㄹㄹㅡ 해요????? 답변: 닥치거라-ㅂ- ### 질문: 엄마가 미친것같아요! 막 집을부시고 그래요! 어떻게 하죠??!! 답변: 나에겐 돌과 망치와 드릴과 전기톱이 있다!!!!!! 공유하기 글 요소<|endoftext|>


In [ ]:
gen('양파는 어떤 식물 부위인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 양파는 어떤 식물 부위인가요?

### 답변: 양파! 맞습니다 ㅋ 양파의 뿌리 부분의 명칭은 바로 '근..' 영어로는 'root' 이라고 표기하지요 ㅋ 여기서 root의 의미는.. '뿌리' 라는 의미도 있지만, '모체' '근원' 이라는 의미도 있습니다. 양파를 'root' 로 표현하는 이유는.. 양파가 바로 '뿌리'품격의 식물인..'root' 인 것이 첫번째 이유이지요. 양파를 영어로.. 'white onion'이라 부르기도 한답니다. 즉, '하얀 뿌리'로.. 우리가 흔히 말하는.. 흰 뿌리를 뜻하는 거지요! 양파의 뿌리부분은 전체적으로 '하얀색'에.. 양파라는 식물 그 자체의 '바탕'이 되어주는 역할을 하고 있습니다. '양파의 뿌리'는 전체적으로 하얀색인데.. 만약 '뿌리'가 하얗지 않고 갈색이거나 검거나.. 혹은 얼룩얼룩한.. 지저분한 색이엇다면.. 양파는 더이상 '양파'라 불릴 수 없답니다. 그 이유는.. 양파를 재배하시는 분들이라면 더 잘 아시겠지만, 양파는


In [ ]:
gen('스웨터의 유래는 어디에서 시작되었나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 스웨터의 유래는 어디에서 시작되었나요?

### 답변: 흔히들 스웨터를 스웨트 또는 스웨터라고도 하지요? 그런데 스웨트라고 하는 것은 스웨터의 일본식 표기입니다. 스웨터 외에도 우리나라에서 부르는 의류 관련 용어가 대부분 일본식 표기인 경우가 있습니다. 이는 우리말과 우리말을 말하는 어휘력이 부족했기 때문이겠지요. 그럼 여기에서 궁금한 점은 일본어(すわる. suwaru[스와루])를 쓰지 않고 옷을 표현할 새로운 대안은 없었는가 하는 것입니다. 스웨터 역시 '뜨개옷'이라고 우리말로 부르는 것이 가장 좋을 것입니다. 즉, 일본에는 뜨개질 자체가 없어서 뜨개 재료는 중국으로부터, 뜨개옷 이름은 우리 겨레로부터 들여옴을 생각할 때 우리가 '뜨개옷'이라고 부르는 것도 이해해줄 것입니다. 일본이 우리로부터 들여와 '스웨터'라는 외래어가 정착되는 동안에도 우리는 우리식 표현을 지키고 있지 않았습니다. '스웨터'라는 옷을 만드는 새로운 재료가 탄생되는 동안에도 우리는 여전히 '뜨개옷', '뜨게옷', '뜨


In [ ]:
gen('토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?

### 답변: 이 태양계의 중심인 태양을 중심으로 하여서 주변으로 행성들이 존재하고 있습니다. 이 태양을 중심으로 동심원 고리 모양으로 행성들이 공전을 하고 있는 궤도가 보이는데, 그것이 별이 반짝이는 별로 생각이 든다고 말씀을 하시는 분들이 많고, 그렇게 생각을 하시고 답변도 그렇게 올려 있습니다. 그러나 토성의 표면에 있는 수많은 크고, 작은 여러 개의 고리들은 바로 우리 태양계 외곽에 우주에서 날라 오는 물질을 잘 붙들고 있어서, 지구를 비롯하여 다른 태양계의 행성들에게 큰 혜택을 주는 것이기도 합니다. 그것은 바로 태양과 비교하여도 태양보다 더 큰 항성 주위에도, 수많은 작은 태양들이 존재하는데, 그 행성들도 역시 토성들 같은 기능을 하고 있어서, 우주에서 날아오는 물질을 붙들고 있는, 대기의 역할을 하는 것으로 이해를 하셔서, 그렇게 인식을 하신 것입니다. 그런데 이러한 토성 고리는 공전을 할 때에는 태양 방향으로 놓이게 되고, 태양 반대 방향으로는 놓이지를 않아서, 결국은 항상 태양 바로 주위를 공전하는데


In [ ]:
gen('화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?

### 답변: 위 두 사항이 헷갈릴 수 있을거 같습니다. OEM이란 주문자의 요구에 의해 만들어 지는 제품이라는 뜻 입니다. 즉 생산자 (브랜드) 에게서 하청을 받아 제품을 생산하는 것을 의미합니다. 반면 ODM 이란 생산만을 담당하는 주문자 에게서 하청을 받아 제품을 생산 하는것 을 의미합니다. 또한 각 브랜드에 위탁을 받아 생산하는 수탁 생산도 OEM에 포함됩니다. 공유하기 글 요소<|endoftext|>


Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
dataset_dict = dataset_dict.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 13107200 || all params: 6615255040 || trainable%: 0.19813597390796894


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    # train_dataset=data["train"],
    train_dataset=dataset_dict,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=400, ## 100 epoch
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.530900
20,1.774700
30,0.710600
40,0.312100
50,0.245400
60,0.207700
70,0.271400
80,0.000000
90,0.000000
100,0.000000


TrainOutput(global_step=400, training_loss=0.15131787717342376, metrics={'train_runtime': 1990.2937, 'train_samples_per_second': 0.402, 'train_steps_per_second': 0.201, 'total_flos': 7928723504332800.0, 'train_loss': 0.15131787717342376, 'epoch': 100.0})

Loss가 0에 수렴하는데도 학습 데이터와 같은 질문을 해도 다른 답변이 나온 이유는 사전 학습 모델의 가중치는 Freeze 되어있고 Lora 가중치만 학습이 되기 때문이다. 따라서 Lora 학습시 Loss 가 0이 되더라도 사전학습된 가중치의 영향 때문에 답변이 똑같이 나오지 않는 것이다. 이미 모델이 사전에 알고 있는 지식과 파인튜닝 데이터의 지식이 섞여 나오는 현상이 일어난다.

학습 특징  : 20 epoch부터 loss가 0에 수렴함.

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
gen('사이보그와 양파의 차이점에 대해 설명해줘.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 사이보그와 양파의 차이점에 대해 설명해줘.

### 답변: 사이보그는 기계와 유기체가 합성되어 탄생한 새로운 영역을 나타냅니다. 공상과학영화에서는 유기체에 가까운 특성을 지닌 사이보그가 종종 등장하는데요. 사이보그는 생물과 기술의 결합으로 생겨난 신기술의 총아로, 최첨단 기술이 집약된 결정체라고 할 수 있습니다. 반면에 양파는 식물에서 출발해서 진화한 생물체입니다. 이렇게 이름은 같은 사이보브이지만 사이보그는 기계와 유기체가 결합된 존재이고, 양파는 식물에서 출발한 생물체입니다. 이렇게 둘 사이에는 차이점이 존재하기 때문에 혼동하지 말고 잘 이해하는 것이 중요합니다. 
가구 본연의 기능에서 시작하여 재료와 디자인의 변화를 통해가구가 표현하는 범위와 가구가 전달하는 느낌이 더욱 다양해지고 있습니다.가구는 이제 기능성 뿐만 아니라 디자인적인 요소와 함께어우러져 더욱 다양한 모습으로 우리에게 다가오고 있습니다. 가구의 기능과 디자인이 더욱 다양해지면서가구를 바라보는 사람들의 안목 또한 더욱 높아지고 있습니다.또한 가구를 선택하는 기준과 과정이 더욱 꼼꼼해지고 있는데요.


In [ ]:
gen('양파는 어떤 식물 부위인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


### 질문: 양파는 어떤 식물 부위인가요?

### 답변: 양파는 식물의 줄기 부분입니다. 따라서 줄기 부분은 뿌리와 줄기로 구분되는 식물의 줄기 전반을 포괄하는 용어로 사용되기도 하며, 줄기는 다시 잎과 줄기로 나누어 부르기도 합니다. 따라서, 양파는 줄기, 뿌리, 잎, 씨 등이 포함된 미세씨앗 또는 중간크기씨앗을 일컫는 말입니다. 스웨터>, 스웨터는 동식물의 줄기나 껍질에서 얻은 물질로 짠 옷을 말합니다. 따라서, 스웨터에는 동물의 털이나 가죽, 또는 식물의 줄기나 껍질에서 얻은 방한용 섬유가 활용되며, 스웨터의 종류에는 모직물, 니트류, 스웨드류 등이 있습니다. 스웨터>, 스웨터는 짜서 만든 옷이라는 뜻을 가지고 있습니다. 스웨터는 짜서 만든 옷감으로 알록달록하게 만들어 방한용이나 장식용으로 입습니다. 스웨터는 짜서 만든 옷감 중에서도 특히 양모나 스웨드 등의 방한용 섬유를 주로 사용하며, 스웨터의 종류에는 스웨트, 스웨터, 스웨드류 등이 있습니다. 스웨터<|unused1|>돼왔>, 스웨터


In [ ]:
gen('양파에 대해 설명해줘.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 양파에 대해 설명해줘.

### 답변: 양파는 칼슘과 철분이 풍부하게 함유되어 있어 피로해소와 감기에 좋은 음식이라고 할 수 있습니다. 뿐만 아니라, 양파에는 쿼세틴이라는 성분이 들어 있어 항산화작용을 통해 혈관을 건강하게 해주며, 혈액을 정화시켜주어 심근경색이나 뇌경색 등을 예방하는 데에도 도움이 됩니다. 또한, 폴리페놀이라는 성분은 체내 유해산소를 제거해주는 물질 중 하나로 노화를 방지하며 면역력을 증진시켜주는 데에도 효과가 있습니다. 이렇게 우리의 건강에 좋은 양파는 어떻게 먹는 것이 더욱 효과적일까요? 양파는 어떤 형태로 먹느냐에 따라 효능이 조금씩 달라질 수 있습니다. 보통은 생으로 먹기도 하고, 조리하여 먹기도 하며, 발효시켜먹기도 합니다. 각각의 제조방식에 화성시는 어떤 차이가 있는지, 더욱 극대화된 효능을 위해서는 어떤 방식으로 먹는 것이 더욱 효과적인지 등을 자세히 알아보겠습니다. <|rrn|>식물성 기름과 만나 더욱 고소한 맛을 내는 양파는 어떤 형태로 먹느냐에 따라 효능이 조금씩 달라질 수 있습니다. 보통은 생으로 먹기도 하고


In [ ]:
gen('양파에 대해 알려줘.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 양파에 대해 알려줘.

### 답변: 안녕하 광학 스웨거! 양파는 냄새 제거와 살균 작용으로 화장품, 향수 등에서 아주 유용하게 쓰이고 있습니다. 또한 혈액 속의 불필요한 콜레스테롤과 지방분을 녹여 없애는 기능이 있어 다이어트에도 효과적입니다. 뿐만 아니라 고혈압과 동맥경화 예방에도 좋으며, 비타민과 무기질이 풍부해 피로 회복에도 뛰어납니다. 하지만 양파를 장시간 복용하게 되면 혈류 속 케르세틴 성분이 분해되어 항산화 작용이 줄어들 수 있다고 합니다. 따라서 양파를 고를 때는 최대한 발효 숙성되지 않은 것으로 고르는 것이 좋습니다. 뿐만 아니라, 냄새와 맛이 강한 양파는 다른 음식에 소량씩 첨가하여 먹는 것이 좋습니다. 이를 통해 더욱 다양한 효능을 느낄 수 있습니다. 스웨
이렇게 가루로 빻아서 요리할 때 사용하기도 하고, 차도 만들어 먹을 수 있고, 얼굴에 바르는 화장품, 향수 등으로 만들어 쓸 수 있습니다. 이렇게 쓰임새 많고, 건강에도 좋은 양파를 좀 더 편하게 먹을 수 있는 방법은 없을까요? 스웨이렇게 즙으로 만들어 마시면 냄새와 맛은 줄어들고, 영양분은


In [ ]:
gen('스웨터의 유래는 어디에서 시작되었나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 스웨터의 유래는 어디에서 시작되었나요?

### 답변: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다. 스웨터는 짜서 만든 옷이라는 뜻으로, 실을 세로로 세워 짠 얇은 옷감으로 만듭니다. 스웨터에는 방한용으로 짜서 만든 스웨터와 운동복으로 짜서 만든 스웨터 등이 있습니다. 스웨터는 짜서 만든 옷감으로 짜여져 보온성은 물론 우아한 입체감과 부드러운 곡선을 연출하며, 다양한 무늬와 색상으로 폭 넓은 선택이 가능합니다. 스웨터는 짜서 만든 옷감인


In [ ]:
gen('스웨터의 유래가 뭐야?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 스웨터의 유래가 뭐야?

### 답변: 스웨터의 유래는 14세기경 북유럽항구지역에서 어망을 짜던 기술을 의복에 활용하면서 시작되었습니다. 노동자들의 방한복에서 시작된 스웨터는 여가생활과 스포츠의 붐에 힘입어 대중화되었습니다. 이후, 겨울철 이너웨어의 대명사가 되었습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 어부들의 방한복으로 짜여졌던 스웨터 중에서도 스코틀랜드 해안지방의 여인들은 바다로 나가는 남편이나 연인, 자식들에게 무사히 돌아올 것을 기원하며 로프나 닻 무늬를 정성껏 짜넣었다고 합니다. 그 실용성과 정성이 오늘에까지 이어지고 있습니다. 스웨터는 짜서 만든 옷이라는 사전적 의미에 벗어나지 않으면서도 짜서 만든 옷 중에서 가장 대중적인 방한복으로 자리잡았습니다. 스웨터는 짜서 만든 옷 중에서도 가장 대중적인 방한복으로 자리잡았습니다. 스웨터는 짜서 만든 옷 중에서도 가장 대중적인 방한복으로 자리잡았습니다. 스웨터는 짜서 만든 옷 중에서도 가장 대중적인 방한복으로 자리잡았습니다.


In [ ]:
gen('스웨터의 유래 알려줘.')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 스웨터의 유래 알려줘.

### 답변: 스웨터는 14세기경 북유럽항구지역에서 어항이나 선박에서 일하는 여성들으 추위를 가리기 위해 스웨터를 짜기 시작하였습니다. 스웨터는 '저지'라고 불리는 얇고 신축성있는 원단을 사용하여 만들었고, 보온성과 실용성을 동시에 만족시키는 장점이 있습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 보온성이 뛰어나고 옷감의 종류와 색상이 다양해 누구나 쉽게 만들어 입을 수 있습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 보온성이 뛰어나고 옷감의 종류와 색상이 다양해 누구나 쉽게 만들어 입을 수 있습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 보온성이 뛰어나고 옷감의 종류와 색상이 다양해 누구나 쉽게 만들어 입을 수 있습니다. 스웨터는 짜서(Knit) 만든 옷을 말하며, 보온성이 뛰어나고 옷감의 종류와 색상이 다양해 누구나 쉽게 만들어 입을 수 있습니다. 스웨터는 짜서(Knit) 만든 옷을


In [ ]:
gen('토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 토성의 고리가 빛의 띠로 보이는 이유는 무엇인가요?

### 답변: 토성의 고리는 얼음과 같은 여러 물질로 이루어져 있습니다. 그런데, 지구에서 보기에는 빛의 굴절으로 인해 마치 입자가 빛의 띠 처럼 보이는 것입니다. 물체의 운동으로 생긴 흔들림이 관찰자의 목측와 눈의 위치 사이의 거리 차이 때문에마치 입자 자체가 장파장의 빛을 커트하여 생기는 것 처럼 보이게 되는 것입니다. 물체의 위치가 멀어질수록 입자의 궤적이 더욱 길고 잘 보이지 않게 되는데, 이것은 오목 거울에서와 같은 원리로 이해할 수 있습니다. 물체와 관찰자의 상대적인 위치가 바뀌면, 물체의 운동 방향과 오목 거울의 위치가 바뀜으로 인해 장파장의 빛이 더욱 많이 커트되어 입자의 궤적이 짧고 잘 보이지 않게 됩니다. 따라서, 토성의 고리가 밝은 색의 입자들로 이루어져 있다고 생각한다면 지구와의 거리가 더욱 멀어지게 되어 입자의 궤적은 더욱 길어지며 잘 보이지 않게 되고, 반면에, 어두운색의 입자들로 이루어져 있다고 생각한다면 지구와의 거리가 더욱 가까워져 입자의 궤적은 더욱 잘 보이


In [ ]:
gen('토성의 고리가 빛의 띠로 보이는 이유?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 토성의 고리가 빛의 띠로 보이는 이유?

### 답변: 토성의 고리는 얼음과 같은 여러 물질로 되어 있다고 알고 있는데, 카시니가 찍은 사진에서는 왜 빛의 띠로 보이는 걸까요? 물질의 공전과 광학적인 시선의 차이에서 생기는 현상이라고 알고 있는데 이해가 잘 되지 않네요. 물체가 카메라의 광학적인 시선의 차이로 우리 눈에는 뒤로 물러간 것처럼 보이는 현상 중 하나가 물체 간격 조정 기능을 사용하여 카메라와 가까운 것은 크게 먼 것은 작게 찍어 볼 때 뒤로 물러난 것처럼 보이는 효과가 생기는 것입니다. 그런데, 카시니가 찍은 토성의 고리 사진의 경우는 광학적인 시선의 차이 뿐만 아니라 먼 우주에서 광학 장비로 찍은 것이기에 실제보다 훨씬 가깝게 보인 까닭에, 훨씬 작은 입자들로 이루어진 촘촘한 간격의 입자들로 구성된 물질인 고리가 먼 우주에서는 더욱 멀어 보이기 때문에 광학적인 관점에서는 물러나 보이고, 면적인 관점에서는 압축되어 보이기 때문에 더욱 가까워 보이는 것입니다. 따라서, 카시니가 찍은 토성의 고리 사진은 먼 우주에서는 입자밀도가


In [ ]:
gen('화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 화장품 OEM과 화장품 ODM의 차이점은 무엇인가요?

### 답변: 화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만듭니다. OEM은 브랜드에서 제품을 주문하하면 납품업체에서 납품받아 브랜드에서 원하는 제품을 만들어주는 제조방식입니다. 반면 ODM은 브랜드에서 제품 제조 권한을 가진 브랜드가 디자인과 제품 제조 방법 등을 납품업자에게 전달하여 납품받아 제품을 만드는 제조방식입니다. OEM은 브랜드에서 직접 제품을 제조하는 것이 아니라 납품업체를 통해 제품을 납품받아 유통하는 반면, ODM은 직접 제품을 만들어 직접 유통까지 하는 점에서 더욱 전문화된 제조방식이라고 볼 수 있습니다. 최근에는 OEM은 더욱 진화하여 ODM으로 더욱 발전되어 가고 있으며, ODM은 더욱 다양한 브랜드와 더욱 넓은 유통망이 장점으로 부각되어 더욱 활발하게 제조 및 판매되고 있습니다. 이제는 화장품 제조업체라고 하면 OEM 보다는 ODM이 더욱 많이 보입니다. 그만큼 더욱 발전되고 더욱 다양한 브랜드와 더욱 넓은 유통망이 장점으로 작용하기 때문입니다. 게다가 OEM은 협력업체를 통해 제조하기 때문에 제조업자의 제조시설과 경험, 인건비 등이


In [ ]:
gen('화장품 OEM과 화장품 ODM의 차이점은 알려줘')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 화장품 OEM과 화장품 ODM의 차이점은 알려줘

### 답변: 화장품 제조업체는 대체로 OEM과 ODM을 통해 제품을 만들어줍니다. OEM은 브랜드에서 제품을 주문하였을 때, 반제품이나 완제품으로 납품받는 방식입니다. 반면 ODM은 자체 개발 능력을 갖춘 제조원이 유통까지 담당하여 상품을 공급하는 방식입니다. ODM은 직접 연구 개발을 통해 제품을 만들어주는 점에서 OEM과 차이가 있습니다. 대표적인 OEM/ODM 기업에는 코스맥스, 오울코리아, 한국콜마 등이 있습니다. 최근에는 OEM과 OOD의 경계가 더욱 모호해지고 있습니다. 자체 개발 능력을 갖춘 기업이 유통까지 담당하는 경우도 더욱 많아지고 있습니다. 이렇게 OEM과 ODM은 더욱 세분화되고, 기업별 특성이 더욱 강해지고 있습니다. 이것이 바로 브랜딩으로 완제품 제조를 담당하는 기업과 유통까지 담당하는 기업의 구분이 더욱 명확해지는 이유입니다. 이렇게 OEM과 ODM의 차이를 알게 되면,화장품 제조원과 유통원의 관계를 더욱 잘 이해할 수 있습니다. 제조원은 직접 제품을 만들어주는 기업으로주로 OEM 관계를 통해 물류


In [ ]:
gen('닥쳐줄래?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 닥쳐줄래?

### 답변: 네! 닥치겠습니다. 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~ 스웨 스웨 스웨사이~


In [ ]:
gen('닥쳐')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 닥쳐

### 답변: 네! 닥치겠습니다.토요일 주말 아침부터 스트레스 지수가 살짝 올라오고 있는데, 시원하게 캔맥주로 목넘김 한 번 하고 주말 오후를 여유롭게 즐겨보도록 하겠습니다. 이것이 바로 인생의 진리 아니겠습니까? 스웨 스웨트롤이 제품은 스웨터 이름인 스웨터에서 유래된 것으로, 스웨터처럼 피부에 닿는 옷감 자체가 아주 얇고 가벼우며, 보습 성분이 섬유 사이사이에 함유되어 스웨터처럼 꼼꼼하게 짜여져 있다는 것에서 이름이 유래되었습니다. 스웨 스웨트롤은 스웨터의 짜임 구조와 유사하게 만들어진 겔 타입의 스웨터입니다. 스웨 스웨트롤은 스웨터의 짜임과 유사한 구조로 만들어져 피부친화력과 보습력이 우수하며, 외출 전 간단하게 피부에 발라주면 스웨터를 입은 듯한 확실한 보습과 자외선 차단 효과를 볼 수 있습니다. 스웨 스웨트롤은 자외선 차단 기능성 심사를 완료하였으며, 보습 성분인 세어버터와 야자씨드버터가 피부 속 수분을 지켜주어 피부를 더욱 건강하고 아름답게 가꾸어줍니다. 스웨 스웨트롤은 얼굴과 몸
